In [1]:
from sleepmate.agent import *
from sleepmate.user import *
from sleepmate.diary import *
from sleepmate.isi import *
from sleepmate.act import *
from sleepmate.seeds import *
from sleepmate.stress import *
from sleepmate.history import *
from sleepmate.goal import *
from sleepmate.act import *
from sleepmate.knowledge import *
from sleepmate.stimulus import *

In [35]:
!rm -f /Users/cck197/Downloads/virtual_greg/memory.pkl

# Database

In [ ]:
DBSeedsDiaryEntry.objects()

In [7]:
DBSeedsDiaryEntry.objects.delete()

1

In [9]:
DBSeedsDiaryEntry.objects.delete()

2

In [8]:
[o.to_mongo() for o in DBSeedsDiaryEntry.objects()]

[SON([('_id', ObjectId('6532c988f1baf405e85f2793')), ('pod', ObjectId('6532c914f1baf405e85f2792')), ('date', datetime.datetime(2023, 10, 20, 0, 0)), ('sleep_1', 0), ('sleep_2', 0), ('sleep_3', 0), ('exercise_1', 0), ('exercise_2', 0), ('exercise_3', 0), ('eating_1', 0), ('eating_2', 0), ('eating_3', 0), ('drinking_1', 0), ('drinking_2', 0), ('drinking_3', 0), ('stress_1', 0), ('stress_2', 0), ('stress_3', 0)]),
 SON([('_id', ObjectId('6532c9c5f1baf405e85f2794')), ('pod', ObjectId('6532c914f1baf405e85f2792')), ('date', datetime.datetime(2023, 10, 20, 0, 0)), ('sleep_1', 1), ('sleep_2', 1), ('sleep_3', 1), ('exercise_1', 1), ('exercise_2', 1), ('exercise_3', 1), ('eating_1', 1), ('eating_2', 1), ('eating_3', 1), ('drinking_1', 1), ('drinking_2', 1), ('drinking_3', 1), ('stress_1', 1), ('stress_2', 1), ('stress_3', 1)])]

In [8]:
DBSleepDiaryEntry.objects.order_by("-id")

[<SleepDiaryEntry: SleepDiaryEntry object>]

In [2]:
DBDailyRoutineSeen.objects.count()

0

In [4]:
save_daily_routine_seen(None, None, None)

save entry={'date': datetime.datetime(2023, 10, 20, 10, 17, 35, 897506)}


In [3]:
DailyRoutineSeen()

DailyRoutineSeen(date=datetime.datetime(2023, 10, 20, 10, 17, 5, 170687))

In [5]:
DBDailyRoutineSeen.objects.delete()

1

In [9]:
DBSleepDiaryEntry.objects.delete()

1

In [2]:
def clear_db():
    return [
        (cls.__name__, cls.objects.delete())
        for cls in [
            DBHealthHistory,
            DBExerciseEntry,
            DBVLQEntry,
            DBSleepDiaryEntry,
            DBISIEntry,
            DBSeedPod,
            DBSeedsDiaryEntry,
            DBStressAudit,
            DBGoalRefusal,
            DBStimulusControlSeen,
            DBDailyRoutineSeen
        ]
    ]

In [3]:
clear_db()

[('HealthHistory', 1),
 ('ExerciseEntry', 0),
 ('VLQEntry', 0),
 ('SleepDiaryEntry', 1),
 ('ISIEntry', 1),
 ('SeedPod', 6),
 ('SeedsDiaryEntry', 1),
 ('StressAudit', 1),
 ('GoalRefusal', 1),
 ('StimulusControlSeen', 1),
 ('DailyRoutineSeen', 1)]

In [4]:
db_user = User(name="Chris", email="chris@nourishbalancethrive.com")

In [5]:
db_user.save()

<User: User object>

In [2]:
# Querying the User Document
db_user = DBUser.objects(email="chris@nourishbalancethrive.com").first()

In [10]:
db_user = get_current_user()

In [5]:
db_user.to_json()

'{"_id": {"$oid": "6526d86e6015be5b3434ac13"}, "name": "Chris", "email": "chris@nourishbalancethrive.com"}'

# VLQ

In [12]:
db_entry = DBVLQEntry(user=db_user, date=datetime.now(), marriage=[5, 5])#.to_mongo()

In [13]:
db_entry.save()

ValidationError: ValidationError (VLQEntry:None) (Field is required: ['family_relations', 'parenting', 'friendships', 'work', 'education', 'recreation', 'spirituality', 'citizenship', 'physical_self_care', 'environmental_issues', 'art'])

In [15]:
from langchain.output_parsers import PydanticOutputParser

In [16]:
parser = PydanticOutputParser(pydantic_object=VLQEntry)

In [18]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"date": {"title": "Date", "description": "date of entry", "type": "string"}, "family_relations": {"title": "Family Relations", "description": "family relations", "type": "array", "minItems": 2, "maxItems": 2, "items": [{"type": "integer"}, {"type": "integer"}]}, "marriage": {"title": "Marriage", "description": "marriage/couples/intimate relations", "type": "array", "minItems": 2, "maxItems": 2, "items": [{"type": "integer"}, {"type": "integer"}]}, "parenting": {"title": "Parenting", "description": "being a parent", "type": "arr

# SEEDS

In [2]:
memory = X.load_memory()

load_memory filename='/Users/cck197/Downloads/virtual_greg/memory.pkl'


In [3]:
db_seed_pod = get_current_seed_pod()

In [4]:
entry = get_seeds_from_memory(ReadOnlySharedMemory(memory=memory))

In [3]:
db_entry = get_current_seeds()

In [4]:
entry = get_seed_pod_from_memory(ReadOnlySharedMemory(memory=memory))

In [4]:
db_entry.to_mongo().to_dict()

{'_id': ObjectId('652c7fcb567f0d4dc33e1676'),
 'pod': ObjectId('652ad74ddf36ed139525fbe3'),
 'date': datetime.datetime(2023, 10, 15, 0, 0),
 'sleep_1': 1,
 'sleep_2': 1,
 'sleep_3': 1,
 'exercise_1': 1,
 'exercise_2': 1,
 'exercise_3': 1,
 'eating_1': 1,
 'eating_2': 1,
 'eating_3': 1,
 'drinking_1': 1,
 'drinking_2': 1,
 'drinking_3': 1,
 'stress_1': 1,
 'stress_2': 1,
 'stress_3': 1}

In [2]:
get_seeds_diary_entry(None, None, None)

get_seeds_diary_entry db_entry=<SeedsDiaryEntry: SeedsDiaryEntry object>


'{"pod": "{\\"sleep_1\\": \\"Be in bed by 9.30 pm\\", \\"sleep_2\\": \\"Turn off my cell phone by 8 pm\\", \\"sleep_3\\": \\"No caffeine after 12 pm\\", \\"exercise_1\\": \\"Walk outside for 10 minutes before 9:30 am\\", \\"exercise_2\\": \\"Do 10 push-ups while the coffee is brewing\\", \\"exercise_3\\": \\"Take the stairs instead of the elevator at work\\", \\"eating_1\\": \\"Eat 50g of protein with every meal\\", \\"eating_2\\": \\"Eat one meal with chopsticks (to slow down eating)\\", \\"eating_3\\": \\"Include a serving of vegetables in each meal\\", \\"drinking_1\\": \\"Drink at least 8 glasses of water a day\\", \\"drinking_2\\": \\"Don\'t drink out of anything plastic\\", \\"drinking_3\\": \\"Drink fizzy water from 6-8 pm (for booze cravings)\\", \\"stress_1\\": \\"Write down three tiny things I\'ve been grateful for today\\", \\"stress_2\\": \\"Do the \'leaves on a stream\' or \'open focus\' exercises\\", \\"stress_3\\": \\"Take 5 minutes each day to meditate or practice deep 

In [11]:
db_entry.pod

<SeedPod: SeedPod object>

In [6]:
save_sleep_diary_entry_to_db??

Signature:
save_sleep_diary_entry_to_db(
    user: mongoengine.base.metaclasses.User,
    entry: sleepmate.diary.SleepDiaryEntry,
) -> mongoengine.base.metaclasses.SleepDiaryEntry
Docstring: <no docstring>
Source:   
def save_sleep_diary_entry_to_db(
    user: DBUser, entry: SleepDiaryEntry
) -> DBSleepDiaryEntry:
    entry = adjust_to_baseline_date(entry.dict())
    # delete any existing entries for this date
    DBSleepDiaryEntry.objects(user=user, date=entry["date"]).delete()
    # save the new entry
    return DBSleepDiaryEntry(**{"user": user, **entry}).save()
File:      ~/Documents/sleepmate/sleepmate/diary.py
Type:      function

In [7]:
save_seed_pod_to_db(get_current_user(), entry)

<SeedPod: SeedPod object>

In [8]:
db_seed_pod = _; db_seed_pod.to_mongo()

SON([('_id', ObjectId('652ad1e25b1a9e51cb0d4463')), ('user', ObjectId('6526d86e6015be5b3434ac13')), ('sleep_1', 'Be in bed by 9.30 pm'), ('sleep_2', 'Turn off my cell phone by 8 pm'), ('sleep_3', 'No caffeine after 12 pm'), ('exercise_1', 'Walk outside for 10 ten minutes before 9:30 am'), ('exercise_2', 'Do 10 push-ups while the coffee is brewing'), ('exercise_3', 'Take the stairs instead of the elevator'), ('eating_1', 'Eat one meal with chopsticks (to slow down eating)'), ('eating_2', 'Eat 50g of protein with every meal'), ('eating_3', 'Include a serving of vegetables in each meal'), ('drinking_1', "Don't drink out of anything plastic"), ('drinking_2', 'Drink fizzy water from 6-8 pm (for booze cravings)'), ('drinking_3', 'Drink at least 2 liters of water daily'), ('stress_1', "Write down three tiny things I've been grateful for today"), ('stress_2', "Do the 'leaves on a stream' or 'open focus' exercises"), ('stress_3', 'Take 5 minutes each day to meditate or practice mindfulness')])

In [7]:
db_seed_pod = DBSeedPod(sleep_1="dim the lights when it gets dark out", user=db_user)

In [8]:
db_seed_pod.save()

<SeedPod: SeedPod object>

In [11]:
DBSeedPod.objects().order_by('-id').first().to_mongo()

SON([('_id', ObjectId('652ad74ddf36ed139525fbe3')), ('user', ObjectId('6526d86e6015be5b3434ac13')), ('sleep_1', 'Be in bed by 9.30 pm'), ('sleep_2', 'Turn off my cell phone by 8 pm'), ('sleep_3', 'No caffeine after 12 pm'), ('exercise_1', 'Walk outside for 10 minutes before 9:30 am'), ('exercise_2', 'Do 10 push-ups while the coffee is brewing'), ('exercise_3', 'Take the stairs instead of the elevator at work'), ('eating_1', 'Eat 50g of protein with every meal'), ('eating_2', 'Eat one meal with chopsticks (to slow down eating)'), ('eating_3', 'Include a serving of vegetables in each meal'), ('drinking_1', 'Drink at least 8 glasses of water a day'), ('drinking_2', "Don't drink out of anything plastic"), ('drinking_3', 'Drink fizzy water from 6-8 pm (for booze cravings)'), ('stress_1', "Write down three tiny things I've been grateful for today"), ('stress_2', "Do the 'leaves on a stream' or 'open focus' exercises"), ('stress_3', 'Take 5 minutes each day to meditate or practice deep breath

In [10]:
db_seed_pod.to_mongo()

SON([('_id', ObjectId('652ac048fb26c915b1cc9e8d')), ('user', ObjectId('6526d86e6015be5b3434ac13')), ('sleep_1', 'dim the lights when it gets dark out'), ('sleep_2', ''), ('sleep_3', ''), ('exercise_1', ''), ('exercise_2', ''), ('exercise_3', ''), ('eating_1', ''), ('eating_2', ''), ('eating_3', ''), ('drinking_1', ''), ('drinking_2', ''), ('drinking_3', ''), ('stress_1', ''), ('stress_2', ''), ('stress_3', '')])

In [2]:
DBSeeds.objects(pod=get_current_seed_pod()).order_by("-id").first()

In [15]:
db_seeds = DBSeeds(pod=db_seed_pod, date="Oct 14", sleep_1=1)

In [16]:
db_seeds.save()

<Seeds: Seeds object>

In [17]:
db_seeds.to_mongo()

SON([('_id', ObjectId('652ac0cefb26c915b1cc9e8e')), ('pod', ObjectId('652ac048fb26c915b1cc9e8d')), ('date', datetime.datetime(2023, 10, 14, 0, 0)), ('sleep_1', 1), ('sleep_2', 0), ('sleep_3', 0), ('exercise_1', 0), ('exercise_2', 0), ('exercise_3', 0), ('eating_1', 0), ('eating_2', 0), ('eating_3', 0), ('drinking_1', 0), ('drinking_2', 0), ('drinking_3', 0), ('stress_1', 0), ('stress_2', 0), ('stress_3', 0)])

In [44]:
DBSeed = pydantic_to_mongoengine(Seed)

In [51]:
db_entry = DBSeed(category="Sausage", seed="dim lights when it gets dark out")

In [52]:
db_entry.save()

ValidationError: ValidationError (Seed:None) (Value must be one of ['Sleep', 'Exercise', 'Eating', 'Drinking', 'Stress']: ['category'])

> /Users/cck197/Library/Caches/pypoetry/virtualenvs/sleepmate-kok1GflL-py3.11/lib/python3.11/site-packages/mongoengine/base/document.py(438)validate()
    436                 pk = self._instance.pk
    437             message = f"ValidationError ({self._class_name}:{pk}) "
--> 438             raise ValidationError(message, errors=errors)
    439 
    440     def to_json(self, *args, **kwargs):



ipdb>  exit


In [50]:
db_entry.to_json()

'{"_id": {"$oid": "652ab89a5028dde8398b00fd"}, "category": "Sleep", "seed": "dim lights when it gets dark out"}'

In [14]:
list(Category.__members__.keys())

['sleep', 'exercise', 'eating', 'drinking', 'stress']

In [34]:
type(datetime)

type

# Meet and Greet

In [3]:
memory = X.load_memory()

load_memory filename='/Users/cck197/Downloads/virtual_greg/memory.pkl'


In [9]:
memory.save_context({"input": f"my name is {user.name}"}, {"output": f"hi {user.name}"})

In [5]:
memory.save_context({"input": f"my email is {user.email}"}, {"output": f"got it"})

In [37]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='my name is Chris'),
  AIMessage(content='hi Chris'),
  HumanMessage(content='hey'),
  AIMessage(content='Hi Chris! How can I assist you today?'),
  HumanMessage(content='Tell me a joke.'),
  AIMessage(content='\n{"setup": "Why did the chicken cross the road?", "punchline": "To get to the other side!"}'),
  HumanMessage(content="what's my name?"),
  AIMessage(content='\n{"setup": "What\'s my name?", "punchline": ""}'),
  HumanMessage(content="what's my name?"),
  AIMessage(content='\n{"setup": "What\'s my name?", "punchline": ""}'),
  HumanMessage(content="what's my name?"),
  AIMessage(content='\n{"name": "Your Name"}'),
  HumanMessage(content="what's the human's name?"),
  AIMessage(content='\n{"name": "John"}'),
  HumanMessage(content="what's the human's name?"),
  AIMessage(content='\n\n{"name": "Chris"}'),
  HumanMessage(content='my email is chris@nourishbalancethrive.com'),
  AIMessage(content='got it')]}

In [59]:
x = X(goals["diary_probe"], memory=memory);

get_agent len(tools)=10


Hello, Chris! How are you feeling today?

# Parsing

In [5]:
from typing import List

from langchain.llms import OpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel, Field, validator

In [6]:
model_name = "text-davinci-003"
temperature = 0.0
model = OpenAI(model_name=model_name, temperature=temperature)

In [7]:
# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field


# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

_input = prompt.format_prompt(query=joke_query)

output = model(_input.to_string())

parser.parse(output)

Joke(setup='Why did the chicken cross the road?', punchline='To get to the other side!')

In [18]:
_input.to_string()

'Answer the user query.\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"setup": {"title": "Setup", "description": "question to set up a joke", "type": "string"}, "punchline": {"title": "Punchline", "description": "answer to resolve the joke", "type": "string"}}, "required": ["setup", "punchline"]}\n```\nTell me a joke.\n'

# Using

In [38]:
# Define your desired data structure.
class UserParser(BaseModel):
    name: str = Field(description="name")
    email: str = Field(description="email address")

In [39]:
# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=UserParser)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\nPrevious conversation:{chat_history}",
    input_variables=["query", "chat_history"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [40]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

In [41]:
chain = LLMChain(
    llm=model,
    prompt=prompt,
    verbose=True,
    memory=memory
)

In [55]:
user_query = "what are the human's details"

In [56]:
output = chain({"query": user_query}); output



> Entering new LLMChain chain...
Prompt after formatting:
Answer the user query.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"title": "Name", "description": "name", "type": "string"}, "email": {"title": "Email", "description": "email address", "type": "string"}}, "required": ["name", "email"]}
```
what are the human's details
Previous conversation:[HumanMessage(content='my name is Chris'), AIMessage(content='hi Chris'), HumanMessage(content='hey'), AIMessage(content='Hi Chris! How can I assist you today?'), HumanMessage(content='Tell me a joke.'), AIMessage(co

{'query': "what are the human's details",
 'chat_history': [HumanMessage(content='my name is Chris'),
  AIMessage(content='hi Chris'),
  HumanMessage(content='hey'),
  AIMessage(content='Hi Chris! How can I assist you today?'),
  HumanMessage(content='Tell me a joke.'),
  AIMessage(content='\n{"setup": "Why did the chicken cross the road?", "punchline": "To get to the other side!"}'),
  HumanMessage(content="what's my name?"),
  AIMessage(content='\n{"setup": "What\'s my name?", "punchline": ""}'),
  HumanMessage(content="what's my name?"),
  AIMessage(content='\n{"setup": "What\'s my name?", "punchline": ""}'),
  HumanMessage(content="what's my name?"),
  AIMessage(content='\n{"name": "Your Name"}'),
  HumanMessage(content="what's the human's name?"),
  AIMessage(content='\n{"name": "John"}'),
  HumanMessage(content="what's the human's name?"),
  AIMessage(content='\n\n{"name": "Chris"}'),
  HumanMessage(content='my email is chris@nourishbalancethrive.com'),
  AIMessage(content='got i

In [57]:
parser.parse(output["text"])

UserParser(name='Chris', email='chris@nourishbalancethrive.com')

In [46]:
get_parsed_output(user_query, memory, UserParser)



> Entering new LLMChain chain...
Prompt after formatting:
Answer the user query.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"title": "Name", "description": "name", "type": "string"}, "email": {"title": "Email", "description": "email address", "type": "string"}}, "required": ["name", "email"]}
```
what are the human's name and email?
Previous conversation:[HumanMessage(content='my name is Chris'), AIMessage(content='hi Chris'), HumanMessage(content='hey'), AIMessage(content='Hi Chris! How can I assist you today?'), HumanMessage(content='Tell me a joke.'), AIMe

UserParser(name='Chris', email='chris@nourishbalancethrive.com')

In [47]:
up = _

In [49]:
up.dict()

{'name': 'Chris', 'email': 'chris@nourishbalancethrive.com'}

In [50]:
def pydantic_to_mongoengine(pydantic_model):
    fields = {}
    for name, field in pydantic_model.__fields__.items():
        # This is a basic example and only handles str type; you would expand this for other types.
        if field.outer_type_ == str:
            fields[name] = StringField(required=field.required)
        # Add other field type conversions as needed
    return type(pydantic_model.__name__, (Document,), fields)


In [52]:
User = pydantic_to_mongoengine(UserParser)

In [54]:
User.__dict__

mappingproxy({'name': <mongoengine.fields.StringField at 0x146da2190>,
              'email': <mongoengine.fields.StringField at 0x1477e3650>,
              '_meta': {'abstract': False,
               'max_documents': None,
               'max_size': None,
               'ordering': [],
               'indexes': [],
               'id_field': 'id',
               'index_background': False,
               'index_opts': None,
               'delete_rules': None,
               'allow_inheritance': None,
               'collection': 'user_parser',
               'index_specs': []},
              '_is_base_cls': False,
              '_is_document': True,
              '_cached_reference_fields': [],
              '_fields': {'name': <mongoengine.fields.StringField at 0x146da2190>,
               'email': <mongoengine.fields.StringField at 0x1477e3650>,
               'id': <mongoengine.base.fields.ObjectIdField at 0x147674a10>},
              '_db_field_map': {'name': 'name', 'email': 'ema

# New

In [1]:
from sleepmate.goal import *

In [2]:
goal = Goal(goal="Your goal is...")

In [3]:
str(goal)

'Your goal is...'

In [4]:
goal

Your goal is...

In [13]:
all_args=['["Male", "Feb 13 1976", "Coder", "7 - 6 with breaks", "No", "Some alcohol, mostly at the weekends", "Cyclist, surfer, lifts weights", "Three eggs and sausages for breakfast, salad and protein for lunch, meat and veg for dinner. 1g protein per lb of bodyweight", "None", "None", "No", "No", "No", "Family, three kids, in a house but sleep in a VW Eurovan", "House", "Very rural and quiet", "Sleep through the night, wake up feeling refreshed", "Getting enough daylight", "Apigenin", "No"]']

In [7]:
import json
from sleepmate.history import *
from sleepmate.db import *

In [13]:
DBHealthHistory.objects.delete()

1

In [31]:
[o.to_mongo() for o in DBHealthHistory.objects()]

[SON([('_id', ObjectId('652feafecaa2804944f10028')), ('user', ObjectId('6526d86e6015be5b3434ac13')), ('date', datetime.datetime(2023, 10, 18, 7, 26, 6, 538000)), ('sex', 'Male'), ('date_of_birth', datetime.datetime(1976, 2, 13, 0, 0)), ('occupation', 'Coder'), ('work_hours', '7 - 6 with breaks'), ('smoking', 'No'), ('alcohol', 'Some, mostly at the weekends'), ('physical_activity', 'Cyclist, surfer, lifts weights'), ('diet', 'Three eggs and sausages for breakfast, salad and protein for lunch, meat and veg for dinner. 1g protein per lb of bodyweight'), ('medical_conditions', 'None'), ('medications', 'None'), ('family_history', 'No'), ('mental_health', 'No'), ('psychological_treatment', 'No'), ('living_with', 'Family, three kids'), ('residence_type', 'House, sleeps in a VW Eurovan'), ('noise_level', 'Very rural and quiet'), ('goal', 'Sleep through the night, wake up feeling refreshed'), ('helpful', 'Getting enough daylight'), ('unhelpful', 'Apigenin'), ('notes', '')])]

In [23]:
args = json.loads(all_args[0]); args

['Male',
 'Feb 13 1976',
 'Coder',
 '7 - 6 with breaks',
 'No',
 'Some alcohol, mostly at the weekends',
 'Cyclist, surfer, lifts weights',
 'Three eggs and sausages for breakfast, salad and protein for lunch, meat and veg for dinner. 1g protein per lb of bodyweight',
 'None',
 'None',
 'No',
 'No',
 'No',
 'Family, three kids, in a house but sleep in a VW Eurovan',
 'House',
 'Very rural and quiet',
 'Sleep through the night, wake up feeling refreshed',
 'Getting enough daylight',
 'Apigenin',
 'No']

In [28]:
len(field_names), len(args)

(21, 20)

In [24]:
field_names = list(HealthHistory.__fields__.keys()); field_names

['date',
 'sex',
 'date_of_birth',
 'occupation',
 'work_hours',
 'smoking',
 'alcohol',
 'physical_activity',
 'diet',
 'medical_conditions',
 'medications',
 'family_history',
 'mental_health',
 'psychological_treatment',
 'living_with',
 'residence_type',
 'noise_level',
 'goal',
 'helpful',
 'unhelpful',
 'notes']

In [26]:
kwargs = {field: arg for field, arg in zip(field_names, args)}; kwargs

{'date': 'Male',
 'sex': 'Feb 13 1976',
 'date_of_birth': 'Coder',
 'occupation': '7 - 6 with breaks',
 'work_hours': 'No',
 'smoking': 'Some alcohol, mostly at the weekends',
 'alcohol': 'Cyclist, surfer, lifts weights',
 'physical_activity': 'Three eggs and sausages for breakfast, salad and protein for lunch, meat and veg for dinner. 1g protein per lb of bodyweight',
 'diet': 'None',
 'medical_conditions': 'None',
 'medications': 'No',
 'family_history': 'No',
 'mental_health': 'No',
 'psychological_treatment': 'Family, three kids, in a house but sleep in a VW Eurovan',
 'living_with': 'House',
 'residence_type': 'Very rural and quiet',
 'noise_level': 'Sleep through the night, wake up feeling refreshed',
 'goal': 'Getting enough daylight',
 'helpful': 'Apigenin',
 'unhelpful': 'No'}

In [2]:
memory = X.load_memory()

load_memory filename='/Users/cck197/Downloads/virtual_greg/memory.pkl'


In [3]:
get_exercise_entry_from_memory(ReadOnlySharedMemory(memory=memory))

ExerciseEntry(date=datetime.datetime(2022, 3, 22, 0, 0), name='Open Focus', notes="The user reported feeling 'not bad' before the exercise. After the exercise, the user did not provide feedback on their feelings.")

In [30]:
get_parsed_output("summarise the last sleep diary entry", memory_, SleepDiaryEntry)

SleepDiaryEntry(date=datetime.datetime(2023, 10, 5, 0, 0), in_bed=datetime.datetime(2023, 10, 12, 21, 0), tried_to_fall_asleep=datetime.datetime(2023, 10, 12, 23, 0), time_to_fall_asleep=30, times_awake=5, time_awake=50, final_wake_up=datetime.datetime(2023, 10, 12, 6, 30), out_of_bed=datetime.datetime(2023, 10, 12, 6, 30), sleep_quality='okay', medications='melatonin', notes='Slept in the van')

In [2]:
date = parse_date("yesterday", default_days=1); date

datetime.datetime(2023, 10, 15, 17, 1, 11, 900416)

In [3]:
(start, end) = get_start_end(date)

In [6]:
DBSleepDiaryEntry.objects(user=get_current_user(), date__gte=start, date__lte=end).first().to_mongo()

SON([('_id', ObjectId('652d7e2a2a9e7e291a9e7f96')), ('user', ObjectId('6526d86e6015be5b3434ac13')), ('date', datetime.datetime(2023, 10, 15, 11, 17, 14, 729000)), ('in_bed', datetime.datetime(2023, 10, 15, 11, 17, 14, 729000)), ('tried_to_fall_asleep', datetime.datetime(2023, 10, 15, 11, 17, 14, 729000)), ('time_to_fall_asleep', 0), ('times_awake', 0), ('time_awake', 0), ('final_wake_up', datetime.datetime(2023, 10, 16, 4, 15)), ('out_of_bed', datetime.datetime(2023, 10, 16, 4, 40)), ('sleep_quality', 'good'), ('medications', 'no'), ('notes', "No, that's all.")])

In [17]:
DBSleepDiaryEntry.objects.first().to_mongo()

SON([('_id', ObjectId('652d7e2a2a9e7e291a9e7f96')), ('user', ObjectId('6526d86e6015be5b3434ac13')), ('date', datetime.datetime(2023, 10, 15, 11, 17, 14, 729000)), ('in_bed', datetime.datetime(2023, 10, 15, 11, 17, 14, 729000)), ('tried_to_fall_asleep', datetime.datetime(2023, 10, 15, 11, 17, 14, 729000)), ('time_to_fall_asleep', 0), ('times_awake', 0), ('time_awake', 0), ('final_wake_up', datetime.datetime(2023, 10, 16, 4, 15)), ('out_of_bed', datetime.datetime(2023, 10, 16, 4, 40)), ('sleep_quality', 'good'), ('medications', 'no'), ('notes', "No, that's all.")])

In [7]:
from sleepmate.isi import *

In [12]:
memory.chat_memory

ChatMessageHistory(messages=[AIMessage(content="what's your name?"), HumanMessage(content='Chris'), AIMessage(content="what's your email?"), HumanMessage(content='chris@nourishbalancethrive.com'), HumanMessage(content='hey'), AIMessage(content="Hello Chris, nice to meet you. I hope you're doing well today. Could you let me know how you've been feeling recently?"), HumanMessage(content='list the dates for all my ISI entries'), AIMessage(content="It appears that you don't have any previous Insomnia Severity Index (ISI) entries. Would now be a good time to complete the ISI questionnaire?"), HumanMessage(content='ok'), AIMessage(content="Great, let's start with the Insomnia Severity Index (ISI) questionnaire for today, October 16, 2023.\n\n1. On a scale from 0 to 4, how would you rate your difficulty falling asleep? Here's the scale for your reference:\n0: None\n1: Mild\n2: Moderate\n3: Severe\n4: Very severe")])

In [13]:
memory.return_messages

True

In [4]:
from copy import deepcopy

In [5]:
memory = deepcopy(memory)

In [6]:
memory.return_messages = False

In [23]:
memory = X.load_memory()

load_memory filename='/Users/cck197/Downloads/virtual_greg/memory.pkl'


In [26]:
len(memory.chat_memory.messages)

160

In [3]:
entry = get_isi_entry_from_memory(ReadOnlySharedMemory(memory=memory))

parse_date date='2023-10-16'


In [4]:
entry 

ISIEntry(date=datetime.datetime(2023, 10, 16, 0, 0), difficulty_falling_asleep=1, difficulty_staying_asleep=1, problems_waking_up_too_early=2, sleep_pattern_satisfaction=3, sleep_problem_noticeability=2, sleep_problem_worry=2, sleep_problem_interference=3)

In [6]:
memory.load_memory_variables({})

{'chat_history': [AIMessage(content="what's your name?"),
  HumanMessage(content='Chris'),
  AIMessage(content="what's your email?"),
  HumanMessage(content='chris@nourishbalancethrive.com'),
  HumanMessage(content='hey'),
  AIMessage(content="Hello Chris, it's nice to connect with you. I hope you are doing well today. Could you share with me how you've been feeling lately, particularly in regard to your sleep?"),
  HumanMessage(content='list the dates for all my ISI entries'),
  AIMessage(content='It appears that there are no previous Insomnia Severity Index (ISI) entries for your account. Would now be a good time to complete an ISI survey?'),
  HumanMessage(content='ok'),
  AIMessage(content="Great, let's start the Insomnia Severity Index (ISI) survey for today, October 16, 2023. \n\n1. On a scale from 0 to 4, how would you rate your difficulty falling asleep? \n(0: None, 1: Mild, 2: Moderate, 3: Severe, 4: Very severe)"),
  HumanMessage(content='1'),
  AIMessage(content='Thank you f

In [11]:
DBSeedsDiaryEntry.objects().first().to_mongo()

SON([('_id', ObjectId('652d9650fae93b4b1ee15ecd')), ('pod', ObjectId('652d94ea00f31cca36f2d53b')), ('date', datetime.datetime(2023, 10, 16, 0, 0)), ('sleep_1', 1), ('sleep_2', 1), ('sleep_3', 0), ('exercise_1', 1), ('exercise_2', 1), ('exercise_3', 0), ('eating_1', 1), ('eating_2', 1), ('eating_3', 0), ('drinking_1', 1), ('drinking_2', 1), ('drinking_3', 0), ('stress_1', 1), ('stress_2', 1), ('stress_3', 0)])

In [2]:
s = '[["Read a book for 15 minutes before bed.", "Keep the bedroom dark and quiet.", "Avoid screens for an hour before bedtime."], ["Take a 15-minute walk after dinner.", "Do a short yoga routine in the morning.", "Try a new sport or physical activity once a week."], ["Include a fruit or vegetable with every meal.", "Try a new healthy recipe once a week.", "Limit processed foods and opt for whole foods."], ["Replace one soda with a glass of water each day.", "Try a new herbal tea instead of coffee.", "Limit alcohol to one drink per day."], ["Practice mindfulness or meditation for 10 minutes each day.", "Write in a journal for 15 minutes each day.", "Take a few minutes each day to do something you enjoy."]]'

In [3]:
import json

In [4]:
json_object = json.loads(s); json_object

[['Read a book for 15 minutes before bed.',
  'Keep the bedroom dark and quiet.',
  'Avoid screens for an hour before bedtime.'],
 ['Take a 15-minute walk after dinner.',
  'Do a short yoga routine in the morning.',
  'Try a new sport or physical activity once a week.'],
 ['Include a fruit or vegetable with every meal.',
  'Try a new healthy recipe once a week.',
  'Limit processed foods and opt for whole foods.'],
 ['Replace one soda with a glass of water each day.',
  'Try a new herbal tea instead of coffee.',
  'Limit alcohol to one drink per day.'],
 ['Practice mindfulness or meditation for 10 minutes each day.',
  'Write in a journal for 15 minutes each day.',
  'Take a few minutes each day to do something you enjoy.']]

In [5]:
from sleepmate.seeds import *

In [7]:
entry = create_from_positional_args(SeedPod, *json_object)

In [8]:
entry

SeedPod(sleep_1='Read a book for 15 minutes before bed.', sleep_2='Keep the bedroom dark and quiet.', sleep_3='Avoid screens for an hour before bedtime.', exercise_1='Take a 15-minute walk after dinner.', exercise_2='Do a short yoga routine in the morning.', exercise_3='Try a new sport or physical activity once a week.', eating_1='Include a fruit or vegetable with every meal.', eating_2='Try a new healthy recipe once a week.', eating_3='Limit processed foods and opt for whole foods.', drinking_1='Replace one soda with a glass of water each day.', drinking_2='Try a new herbal tea instead of coffee.', drinking_3='Limit alcohol to one drink per day.', stress_1='Practice mindfulness or meditation for 10 minutes each day.', stress_2='Write in a journal for 15 minutes each day.', stress_3='Take a few minutes each day to do something you enjoy.')

In [18]:
%pdb

Automatic pdb calling has been turned ON


In [21]:
# Usage
instance = create_from_positional_args(SleepDiaryEntry, *args)

parse_date date='2021-10-01'
parse_date date='21:15'
parse_date date='21:15'
parse_date date='4:15'
parse_date date='4:40'


In [22]:
instance

SleepDiaryEntry(date=datetime.datetime(2021, 10, 1, 0, 0), in_bed=datetime.datetime(2023, 10, 16, 21, 15), tried_to_fall_asleep=datetime.datetime(2023, 10, 16, 21, 15), time_to_fall_asleep=5, times_awake=0, time_awake=0, final_wake_up=datetime.datetime(2023, 10, 16, 4, 15), out_of_bed=datetime.datetime(2023, 10, 16, 4, 40), sleep_quality='good', medications='no', notes='slept in the van')

In [14]:
SleepDiaryEntry.parse_obj(args)

ValidationError: 1 validation error for SleepDiaryEntry
__root__
  SleepDiaryEntry expected dict not list (type=type_error)

In [8]:
ConversationBufferWindowMemory(chat_memory=memory, memory_key="chat_history")

ValidationError: 1 validation error for ConversationBufferWindowMemory
chat_memory
  instance of BaseChatMessageHistory expected (type=type_error.arbitrary_type; expected_arbitrary_type=BaseChatMessageHistory)

In [13]:
new_memory = ConversationBufferWindowMemory(memory_key="chat_history")

In [17]:
ro_memory = ReadOnlySharedMemory(memory=memory)

In [24]:
ro_memory.memory.chat_memory.messages

[AIMessage(content="what's your name?"),
 HumanMessage(content='Chris'),
 AIMessage(content="what's your email?"),
 HumanMessage(content='chris@nourishbalancethrive.com'),
 HumanMessage(content='hey'),
 AIMessage(content="Hello Chris, it's nice to connect with you. I hope you are doing well today. Could you share with me how you've been feeling lately, particularly in regard to your sleep?"),
 HumanMessage(content='list the dates for all my ISI entries'),
 AIMessage(content='It appears that there are no previous Insomnia Severity Index (ISI) entries for your account. Would now be a good time to complete an ISI survey?'),
 HumanMessage(content='ok'),
 AIMessage(content="Great, let's start the Insomnia Severity Index (ISI) survey for today, October 16, 2023. \n\n1. On a scale from 0 to 4, how would you rate your difficulty falling asleep? \n(0: None, 1: Mild, 2: Moderate, 3: Severe, 4: Very severe)"),
 HumanMessage(content='1'),
 AIMessage(content='Thank you for your response. \n\n2. On 

In [25]:
new_memory.chat_memory.messages

[]

In [26]:
new_memory.chat_memory.messages = ro_memory.memory.chat_memory.messages[-new_memory.k:]

In [27]:
new_memory

ConversationBufferWindowMemory(chat_memory=ChatMessageHistory(messages=[AIMessage(content="Great, let's start the Insomnia Severity Index (ISI) survey for today, October 16, 2023. \n\n1. On a scale from 0 to 4, how would you rate your difficulty falling asleep? \n(0: None, 1: Mild, 2: Moderate, 3: Severe, 4: Very severe)"), HumanMessage(content='1'), AIMessage(content='Thank you for your response. \n\n2. On the same scale, how would you rate your difficulty staying asleep? \n(0: None, 1: Mild, 2: Moderate, 3: Severe, 4: Very severe)'), HumanMessage(content='1'), AIMessage(content='Thank you for your response. \n\n3. How would you rate your problems with waking up too early? \n(0: None, 1: Mild, 2: Moderate, 3: Severe, 4: Very severe)')]), memory_key='chat_history')

In [4]:
entry

ISIEntry(date=datetime.datetime(2023, 10, 16, 0, 0), difficulty_falling_asleep=1, difficulty_staying_asleep=1, problems_waking_up_too_early=2, sleep_pattern_satisfaction=2, sleep_problem_noticeability=1, sleep_problem_worry=1, sleep_problem_interference=1)

## Working

Answer the user query.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"date": {"title": "Date", "description": "date of entry", "type": "string"}, "in_bed": {"title": "In Bed", "description": "time you went to bed", "type": "string"}, "tried_to_fall_asleep": {"title": "Tried To Fall Asleep", "description": "time you tried to fall asleep", "type": "string"}, "time_to_fall_asleep": {"title": "Time To Fall Asleep", "description": "how long it took you to fall asleep", "type": "integer"}, "times_awake": {"title": "Times Awake", "description": "how many times you woke up during the night", "type": "integer"}, "time_awake": {"title": "Time Awake", "description": "total time you were awake during the night", "type": "integer"}, "final_wake_up": {"title": "Final Wake Up", "description": "final wake up time", "type": "string"}, "out_of_bed": {"title": "Out Of Bed", "description": "time you got out of bed", "type": "string"}, "sleep_quality": {"title": "Sleep Quality", "description": "sleep quality", "type": "string"}, "medications": {"title": "Medications", "description": "medications or aids you used to help you sleep", "type": "string"}, "notes": {"title": "Notes", "description": "any other notes you'd like to add", "type": "string"}}, "required": ["date", "in_bed", "tried_to_fall_asleep", "time_to_fall_asleep", "times_awake", "time_awake", "final_wake_up", "out_of_bed", "sleep_quality", "medications", "notes"]}
```
summarise the last sleep diary entry
Previous conversation:[HumanMessage(content='hey'), AIMessage(content="Hello! I'm here to help you record your sleep diary. Before we begin, may I know your name, please?"), HumanMessage(content='Chris'), AIMessage(content='Nice to meet you, Chris! I also need your email address for our records. Could you please provide that?'), HumanMessage(content='chris@nourishbalancethrive.com'), AIMessage(content='Thank you, Chris. Now, is it a good time for you to record a sleep diary entry?'), HumanMessage(content='Oct 7'), AIMessage(content="Great, we'll record the sleep diary entry for October 7th. Let's start with the first question. What time did you go to bed on October 7th?"), HumanMessage(content='21:00'), AIMessage(content='Thank you, Chris. Now, what time did you try to fall asleep?'), HumanMessage(content='21:00'), AIMessage(content='Alright. How long did it take you to fall asleep, in minutes?'), HumanMessage(content='5'), AIMessage(content='Got it. How many times did you wake up during the night?'), HumanMessage(content='1'), AIMessage(content='I see. And how long were you awake during the night, in minutes?'), HumanMessage(content='5'), AIMessage(content='Noted. What was your final wake up time?'), HumanMessage(content='6:30'), AIMessage(content='Thank you. And what time did you get out of bed?'), HumanMessage(content='6:30'), AIMessage(content='Alright. How would you rate your sleep quality? You can choose from "very good", "good", "okay", "bad", "very bad".'), HumanMessage(content='okay'), AIMessage(content='Okay. Did you use any medications or aids to help you sleep?'), HumanMessage(content='melatonin, and for the notes I slept in the van 3'), AIMessage(content="Thank you for sharing that, Chris. Let's summarize:\n\n- Date of entry: October 7th\n- Time you went to bed: 21:00\n- Time you tried to fall asleep: 21:00\n- Time it took to fall asleep: 5 minutes\n- Number of times you woke up during the night: 1\n- Total time you were awake during the night: 5 minutes\n- Final wake up time: 6:30\n- Time you got out of bed: 6:30\n- Sleep quality: Okay\n- Sleep aids: Melatonin\n- Notes: Slept in the van 3\n\nYour sleep efficiency, which is the percentage of time spent asleep while in bed, is 96.7%. Anything over 85% is considered normal.\n\nIs this information correct?"), HumanMessage(content='yep!'), AIMessage(content='Great! Thank you for confirming, Chris. Your sleep diary entry has been successfully recorded. If you have any other questions or need further assistance, feel free to ask. 🚀🚀🚀'), HumanMessage(content='hey'), AIMessage(content='Hello again, Chris! How are you feeling today? Is it a good time to record another sleep diary entry?'), HumanMessage(content='chris@nourishbalancethrive.com'), AIMessage(content='Thank you, Chris. Now, is it a good time for you to record a sleep diary entry?'), HumanMessage(content='yes'), AIMessage(content="Great! Let's record the sleep diary entry for October 10th. What time did you go to bed?"), HumanMessage(content='actually no Oct 1'), AIMessage(content="Alright, we'll record the sleep diary entry for October 1st. What time did you go to bed on that day?"), HumanMessage(content='21:00'), AIMessage(content='Thank you, Chris. Now, what time did you try to fall asleep?'), HumanMessage(content='120'), AIMessage(content="I'm sorry, but the time you provided seems to be incorrect. Could you please provide the time in a 24-hour format? For example, if it's 1:20 AM, you should enter it as 01:20."), HumanMessage(content='23:00'), AIMessage(content='Got it. How long did it take you to fall asleep, in minutes?'), HumanMessage(content='30'), AIMessage(content='Noted. How many times did you wake up during the night?'), HumanMessage(content='5'), AIMessage(content='I see. And how long were you awake during the night, in minutes?'), HumanMessage(content='50'), AIMessage(content='Thank you for that information. What was your final wake up time?'), HumanMessage(content='6:30'), AIMessage(content='Thank you. And what time did you get out of bed?'), HumanMessage(content='6:30'), AIMessage(content='Alright. How would you rate your sleep quality? You can choose from "very good", "good", "okay", "bad", "very bad".'), HumanMessage(content='okay'), AIMessage(content='Okay. Did you use any medications or aids to help you sleep?'), HumanMessage(content='melatonin, and for the notes I slept in the van 4'), AIMessage(content="Thank you for sharing that, Chris. Let's summarize:\n\n- Date of entry: October 1st\n- Time you went to bed: 21:00\n- Time you tried to fall asleep: 23:00\n- Time it took to fall asleep: 30 minutes\n- Number of times you woke up during the night: 5\n- Total time you were awake during the night: 50 minutes\n- Final wake up time: 6:30\n- Time you got out of bed: 6:30\n- Sleep quality: Okay\n- Sleep aids: Melatonin\n- Notes: Slept in the van 4\n\nYour sleep efficiency, which is the percentage of time spent asleep while in bed, is 85.4%. Anything over 85% is considered normal.\n\nIs this information correct?"), HumanMessage(content='yep!'), AIMessage(content='Great! Thank you for confirming, Chris. Your sleep diary entry has been successfully recorded. If you have any other questions or need further assistance, feel free to ask. 🚀🚀🚀')]

{"date": "October 1st", "in_bed": "21:00", "tried_to_fall_asleep": "23:00", "time_to_fall_asleep": 30, "times_awake": 5, "time_awake": 50, "final_wake_up": "6:30", "out_of_bed": "6:30", "sleep_quality": "okay", "medications": "melatonin", "notes": "Slept in the van 4"}

Not working

Answer the user query.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"date": {"title": "Date", "description": "date of entry", "type": "string"}, "in_bed": {"title": "In Bed", "description": "time you went to bed", "type": "string"}, "tried_to_fall_asleep": {"title": "Tried To Fall Asleep", "description": "time you tried to fall asleep", "type": "string"}, "time_to_fall_asleep": {"title": "Time To Fall Asleep", "description": "how long it took you to fall asleep", "type": "integer"}, "times_awake": {"title": "Times Awake", "description": "how many times you woke up during the night", "type": "integer"}, "time_awake": {"title": "Time Awake", "description": "total time you were awake during the night", "type": "integer"}, "final_wake_up": {"title": "Final Wake Up", "description": "final wake up time", "type": "string"}, "out_of_bed": {"title": "Out Of Bed", "description": "time you got out of bed", "type": "string"}, "sleep_quality": {"title": "Sleep Quality", "description": "sleep quality", "type": "string"}, "medications": {"title": "Medications", "description": "medications or aids you used to help you sleep", "type": "string"}, "notes": {"title": "Notes", "description": "any other notes you'd like to add", "type": "string"}}, "required": ["date", "in_bed", "tried_to_fall_asleep", "time_to_fall_asleep", "times_awake", "time_awake", "final_wake_up", "out_of_bed", "sleep_quality", "medications", "notes"]}
```
summarise the last sleep diary entry
Previous conversation:[AIMessage(content="what's your email?"), HumanMessage(content='chris@nourishbalancethrive.com'), HumanMessage(content='okay'), AIMessage(content='I see. How would you rate your sleep quality on a scale of "very good", "good", "okay", "bad", "very bad"?'), AIMessage(content="what's your name?"), HumanMessage(content='Chris'), AIMessage(content="what's your email?"), HumanMessage(content='chris@nourishbalancethrive.com'), HumanMessage(content='melatonin, and for the notes I slept in the van 5'), AIMessage(content='Thanks for sharing that, Chris. Here\'s the summary of your sleep diary entry for October 4, 2023:\n\n- You got in bed at 21:00.\n- You tried to fall asleep at 23:00.\n- It took you 30 minutes to fall asleep.\n- You woke up 5 times during the night.\n- You were awake for a total of 50 minutes during the night.\n- You finally woke up at 06:30.\n- You got out of bed at 06:30.\n- You rated your sleep quality as "okay".\n- You took melatonin as medication.\n- You noted that you "Slept in the van 5".\n\nYour sleep efficiency was 64.91%. Remember, anything over 85% is considered normal. Is this information correct? 🚀🚀🚀'), AIMessage(content="what's your name?"), HumanMessage(content='Chris'), AIMessage(content="what's your email?"), HumanMessage(content='chris@nourishbalancethrive.com'), HumanMessage(content='yep!'), AIMessage(content="Great! I'm glad we could record your sleep diary entry for October 4, 2023. Remember, keeping a regular sleep diary can help identify patterns and areas for improvement in your sleep habits. Keep up the good work! 🚀🚀🚀"), AIMessage(content="what's your name?"), HumanMessage(content='Chris'), AIMessage(content="what's your email?"), HumanMessage(content='chris@nourishbalancethrive.com'), HumanMessage(content='save that entry to the database'), AIMessage(content="I'm sorry, but as an AI, I don't have the ability to directly interact with databases. However, I can help you format your data in a way that's ready to be input into a database. Here's your sleep diary entry in a database-friendly format:\n\n- Date: October 4, 2023\n- Bedtime: 21:00\n- Sleep Attempt Time: 23:00\n- Time to Fall Asleep: 30 minutes\n- Number of Awakenings: 5\n- Total Awake Time: 50 minutes\n- Final Wake Time: 06:30\n- Out of Bed Time: 06:30\n- Sleep Quality: Okay\n- Medications: Melatonin\n- Notes: Slept in the van 5\n\nPlease let me know if you need help with anything else. 🚀🚀🚀"), AIMessage(content="what's your name?"), HumanMessage(content='Chris'), AIMessage(content="what's your email?"), HumanMessage(content='chris@nourishbalancethrive.com'), HumanMessage(content='hey'), AIMessage(content="Hello Chris, nice to meet you. I'm your AI clinician. I'm going to help you with your sleep diary. If you have any sleep diary entries, please share them with me."), AIMessage(content="what's your name?"), HumanMessage(content='Chris'), AIMessage(content="what's your email?"), HumanMessage(content='chris@nourishbalancethrive.com'), HumanMessage(content="what's my email?"), AIMessage(content="Your email is chris@nourishbalancethrive.com. Now, let's move on to the sleep diary. Is now a good time to record a diary entry?"), AIMessage(content="what's your name?"), HumanMessage(content='Chris'), AIMessage(content="what's your email?"), HumanMessage(content='chris@nourishbalancethrive.com'), HumanMessage(content='Oct 5'), AIMessage(content="I'm sorry, but I couldn't find a sleep diary entry for October 5, 2023. Would you like to create a new entry for this date?"), AIMessage(content="what's your name?"), HumanMessage(content='Chris'), AIMessage(content="what's your email?"), HumanMessage(content='chris@nourishbalancethrive.com'), HumanMessage(content='yep'), AIMessage(content="Great! Let's start with the time you went to bed on October 5, 2023."), AIMessage(content="what's your name?"), HumanMessage(content='Chris'), AIMessage(content="what's your email?"), HumanMessage(content='chris@nourishbalancethrive.com'), HumanMessage(content='21:00'), AIMessage(content='Thanks, Chris. Now, what time did you try to fall asleep?'), AIMessage(content="what's your name?"), HumanMessage(content='Chris'), AIMessage(content="what's your email?"), HumanMessage(content='chris@nourishbalancethrive.com'), HumanMessage(content='23:00'), AIMessage(content='Got it. How long did it take you to fall asleep?'), AIMessage(content="what's your name?"), HumanMessage(content='Chris'), AIMessage(content="what's your email?"), HumanMessage(content='chris@nourishbalancethrive.com'), HumanMessage(content='30'), AIMessage(content='Thanks for sharing that. How many times did you wake up during the night?'), AIMessage(content="what's your name?"), HumanMessage(content='Chris'), AIMessage(content="what's your email?"), HumanMessage(content='chris@nourishbalancethrive.com'), HumanMessage(content='5'), AIMessage(content='I see. And how long were you awake during these times? Please provide the total time in minutes.'), AIMessage(content="what's your name?"), HumanMessage(content='Chris'), AIMessage(content="what's your email?"), HumanMessage(content='chris@nourishbalancethrive.com'), HumanMessage(content='50'), AIMessage(content='Thanks for that information. What time did you finally wake up?'), AIMessage(content="what's your name?"), HumanMessage(content='Chris'), AIMessage(content="what's your email?"), HumanMessage(content='chris@nourishbalancethrive.com'), HumanMessage(content='6:30'), AIMessage(content='Thanks for sharing that. And what time did you get out of bed?'), AIMessage(content="what's your name?"), HumanMessage(content='Chris'), AIMessage(content="what's your email?"), HumanMessage(content='chris@nourishbalancethrive.com'), HumanMessage(content='okay'), AIMessage(content='I see. How would you rate your sleep quality on a scale of "very good", "good", "okay", "bad", "very bad"?'), AIMessage(content="what's your name?"), HumanMessage(content='Chris'), AIMessage(content="what's your email?"), HumanMessage(content='chris@nourishbalancethrive.com'), HumanMessage(content='melatonin, and for the notes I slept in the van'), AIMessage(content='Thanks for sharing that, Chris. Here\'s the summary of your sleep diary entry for October 5, 2023:\n\n- You got in bed at 21:00.\n- You tried to fall asleep at 23:00.\n- It took you 30 minutes to fall asleep.\n- You woke up 5 times during the night.\n- You were awake for a total of 50 minutes during the night.\n- You finally woke up at 06:30.\n- You got out of bed at 06:30.\n- You rated your sleep quality as "okay".\n- You took melatonin as medication.\n- You noted that you "Slept in the van".\n\nYour sleep efficiency was 64.91%. Remember, anything over 85% is considered normal. Is this information correct?'), AIMessage(content="what's your name?"), HumanMessage(content='Chris'), AIMessage(content="what's your email?"), HumanMessage(content='chris@nourishbalancethrive.com'), HumanMessage(content='yep!'), AIMessage(content="Great! I've saved your sleep diary entry for October 5, 2023. Remember, keeping a regular sleep diary can help identify patterns and areas for improvement in your sleep habits. Keep up the good work! 🚀🚀🚀")]

{"properties": {"date": {"title": "Date", "description": "date of entry", "type": "string"}, "in_bed": {"title": "In Bed", "description": "time you went to bed", "type": "string"}, "tried_to_fall_asleep": {"title": "Tried To Fall Asleep", "description": "time you tried to fall asleep", "type": "string"}, "time_to_fall_asleep": {"title": "Time To Fall Asleep", "description": "how long it took you to fall asleep", "type": "integer"}, "times_awake": {"title": "Times Awake", "description": "how many times you woke up during the night", "type": "integer"}, "time_awake": {"title": "Time Awake", "description": "total time you were awake during the night", "type": "integer"}, "final_wake_up": {"title": "Final Wake Up", "description": "final wake up time", "type": "string"}, "out_of_bed": {"title": "Out Of Bed", "description": "time you got out of bed

In [5]:
user = get_user(memory); user

User(name='Chris', email='chris@nourishbalancethrive.com')

In [3]:
entry = get_sleep_diary_entry(memory); entry

OutputParserException: Failed to parse SleepDiaryEntry from completion 

The output should be formatted as a JSON instance that conforms to the following schema:

{"properties": {"date": {"title": "Date", "description": "date of entry", "type": "string"}, "in_bed": {"title": "In Bed", "description": "time you went to bed", "type": "string"}, "tried_to_fall_asleep": {"title": "Tried To Fall Asleep", "description": "time you tried to fall asleep", "type": "string"}, "time_to_fall_asleep": {"title": "Time To Fall Asleep", "description": "how long it took you to fall asleep", "type": "integer"}, "times_awake": {"title": "Times Awake", "description": "how many times you woke up during the night", "type": "integer"}, "time_awake": {"title": "Time Awake", "description": "total time you were awake during the night", "type": "integer"}, "final_wake_up": {"title": "Final Wake Up", "description": "final wake up time", "type": "string"}, "out_of_bed. Got: Expecting ',' delimiter: line 1 column 776

In [5]:
db_entry = save_sleep_diary_entry(db_user, entry)

In [6]:
db_entry.to_mongo().to_dict()

{'_id': ObjectId('652738d055cc9d6f13c11cf5'),
 'user': ObjectId('6526d86e6015be5b3434ac13'),
 'date': datetime.datetime(2023, 10, 1, 0, 0),
 'in_bed': datetime.datetime(2023, 10, 1, 21, 0),
 'tried_to_fall_asleep': datetime.datetime(2023, 10, 1, 23, 0),
 'time_to_fall_asleep': 30,
 'times_awake': 5,
 'time_awake': 50,
 'final_wake_up': datetime.datetime(2023, 10, 2, 6, 30),
 'out_of_bed': datetime.datetime(2023, 10, 2, 6, 30),
 'sleep_quality': 'okay',
 'medications': 'melatonin',
 'notes': 'Slept in the van 4'}

In [21]:
DBSleepDiaryEntry.objects.delete()

7

In [44]:
DBSleepDiaryEntry.objects().order_by('-id').first().to_mongo().to_dict()

{'_id': ObjectId('65270f56f2031c8434501e59'),
 'user': ObjectId('6526d86e6015be5b3434ac13'),
 'date': datetime.datetime(2023, 10, 11, 0, 0),
 'in_bed': datetime.datetime(2023, 10, 11, 21, 15),
 'tried_to_fall_asleep': datetime.datetime(2023, 10, 11, 21, 15),
 'time_to_fall_asleep': 5,
 'times_awake': 1,
 'time_awake': 5,
 'final_wake_up': datetime.datetime(2023, 10, 12, 6, 15),
 'out_of_bed': datetime.datetime(2023, 10, 12, 6, 15),
 'sleep_quality': 'Good',
 'medications': 'Melatonin',
 'notes': 'Slept in the van, it was raining'}

In [18]:
DBUser.objects.delete()

1

In [5]:
DBExerciseEntry.objects.delete()

2

In [13]:
date_fields

['date', 'in_bed', 'tried_to_fall_asleep', 'final_wake_up', 'out_of_bed']

In [8]:
sleep_data = entry.dict()

In [9]:
adjust_to_baseline_date(sleep_data)

{'date': datetime.datetime(2023, 10, 1, 0, 0),
 'in_bed': datetime.datetime(2023, 10, 1, 21, 0),
 'tried_to_fall_asleep': datetime.datetime(2023, 10, 1, 23, 0),
 'time_to_fall_asleep': 30,
 'times_awake': 5,
 'time_awake': 50,
 'final_wake_up': datetime.datetime(2023, 10, 1, 6, 30),
 'out_of_bed': datetime.datetime(2023, 10, 1, 6, 30),
 'sleep_quality': 'okay',
 'medications': 'melatonin',
 'notes': 'Slept in the van 4'}

In [14]:
normalized_data = make_monotonic(sleep_data); normalized_data

in_bed
tried_to_fall_asleep
final_wake_up
out_of_bed


{'date': datetime.datetime(2023, 10, 7, 0, 0),
 'in_bed': datetime.datetime(2023, 10, 11, 21, 15),
 'tried_to_fall_asleep': datetime.datetime(2023, 10, 12, 21, 10),
 'final_wake_up': datetime.datetime(2023, 10, 12, 23, 15),
 'out_of_bed': datetime.datetime(2023, 10, 13, 6, 15),
 'sleep_quality': 'Good',
 'medications': 'Melatonin',
 'notes': 'Slept in the van, it was raining'}

In [16]:
date = date_parser("2023-09-01")

In [22]:
DBSleepDiaryEntry.objects(user=db_user, date=date)

[<SleepDiaryEntry: SleepDiaryEntry object>]

In [5]:
[e.date for e in DBSleepDiaryEntry.objects()]

[datetime.datetime(2023, 9, 1, 0, 0),
 datetime.datetime(2023, 9, 2, 0, 0),
 datetime.datetime(2023, 9, 3, 0, 0),
 datetime.datetime(2023, 9, 4, 0, 0),
 datetime.datetime(2023, 9, 5, 0, 0)]

In [6]:
DBSleepDiaryEntry.objects(user=db_user, date=date_parser("2023-10-01")).first().to_mongo()

SON([('_id', ObjectId('652738d055cc9d6f13c11cf5')), ('user', ObjectId('6526d86e6015be5b3434ac13')), ('date', datetime.datetime(2023, 10, 1, 0, 0)), ('in_bed', datetime.datetime(2023, 10, 1, 21, 0)), ('tried_to_fall_asleep', datetime.datetime(2023, 10, 1, 23, 0)), ('time_to_fall_asleep', 30), ('times_awake', 5), ('time_awake', 50), ('final_wake_up', datetime.datetime(2023, 10, 2, 6, 30)), ('out_of_bed', datetime.datetime(2023, 10, 2, 6, 30)), ('sleep_quality', 'okay'), ('medications', 'melatonin'), ('notes', 'Slept in the van 4')])

In [2]:
DBISIEntry.objects()

[<ISIEntry: ISIEntry object>]